In [ ]:
from urllib.request import urlopen
import re
import pandas as pd
from bs4 import BeautifulSoup

url = f"https://race.netkeiba.com/race/shutuba.html?race_id=202405040811"
html = urlopen(url).read()

soup = BeautifulSoup(html)

SyntaxError: invalid syntax (3264894234.py, line 1)

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--headless") #バックグラウンドで実行
driver_path = ChromeDriverManager().install()
popularity_list = []
service = Service(driver_path)

with webdriver.Chrome(service=service, options=options) as driver:
    url = f"https://race.netkeiba.com/race/shutuba.html?race_id=202405040811"
    driver.get(url)
    i = 0
    elements = []
    # for i in range(15):
        # if i < 9:
        #     li_list = driver.find_elements(By.ID, f"ninki-1_0{i+1}")
        # else:
        #     li_list = driver.find_elements(By.ID, f"ninki-1_{i+1}")
    for i in range(1, 16):
        element_id = f"ninki-1_{i:02d}"  # 01から15までのIDを生成
        elements.extend(driver.find_elements(By.ID, element_id))
    for element in elements:
        popularity_list.append(element.text)

driver.quit()


In [6]:
popularity_list

['4',
 '12',
 '13',
 '7',
 '10',
 '5',
 '3',
 '9',
 '8',
 '11',
 '6',
 '1',
 '14',
 '2',
 '15']

In [7]:
soup = BeautifulSoup(html, "html.parser").find("table", class_="Shutuba_Table RaceTable01 ShutubaTable")
df = pd.read_html(html)[0]
df.columns = df.columns.get_level_values(1)
df = df.drop(columns=['印'])
df = df.drop(columns=['Unnamed: 9_level_1'])
df = df.drop(columns=['登録'])
df = df.drop(columns=['メモ'])



In [8]:
df['人気'] = popularity_list

In [157]:
df

,枠,馬 番,馬名,性齢,斤量,騎手,厩舎,馬体重 (増減),人気
0,1,1,ベラジオオペラ,牡4,58.0,横山和生,栗東上村洋行,NaN,4
1,2,2,マテンロウスカイ,セ5,58.0,横山典弘,栗東松永幹夫,NaN,12
2,2,3,ステラヴェローチェ,牡6,58.0,佐々木大輔,栗東須貝尚介,NaN,13
3,3,4,タスティエーラ,牡4,58.0,松山弘平,美浦堀宣行,NaN,7
4,3,5,ノースブリッジ,牡6,58.0,岩田康誠,美浦奥村武,NaN,10
5,4,6,ソールオリエンス,牡4,58.0,横山武史,美浦手塚貴久,NaN,5
6,4,7,ドウデュース,牡5,58.0,武豊,栗東友道康夫,NaN,3
7,5,8,キングズパレス,牡5,58.0,Ａ．シュタルケ,美浦戸田博文,NaN,9
8,5,9,ホウオウビスケッツ,牡4,58.0,岩田望来,美浦奥村武,NaN,8
9,6,10,ダノンベルーガ,牡5,58.0,Ｃ．デムーロ,美浦堀宣行,NaN,11


In [9]:
# horse_id列追加
a_list = soup.find_all("a", href=re.compile(r"/horse/"))
horse_id_list = []
for a in a_list:
    horse_id = re.findall(r"\d{10}", a["href"])[0]
    horse_id_list.append(horse_id)
df["horse_id"] = horse_id_list

# jockey_id列追加
a_list = soup.find_all("a", href=re.compile(r"/jockey/"))
jockey_id_list = []
for a in a_list:
    jockey_id = re.findall(r"\d{5}", a["href"])[0]
    jockey_id_list.append(jockey_id)
df["jockey_id"] = jockey_id_list

# trainer_id列追加
a_list = soup.find_all("a", href=re.compile(r"/trainer/"))
trainer_id_list = []
for a in a_list:
    trainer_id = re.findall(r"\d{5}", a["href"])[0]
    trainer_id_list.append(trainer_id)
df["trainer_id"] = trainer_id_list



In [11]:
df.columns = df.columns.str.replace(" ", "")
df

,枠,馬番,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),人気,horse_id,jockey_id,trainer_id,sex,age,popularity,impost,wakuban
0,1,1,ベラジオオペラ,牡4,58.0,横山和生,栗東上村洋行,NaN,4,2020102781,01140,01168,0,4,4,58.0,1
1,2,2,マテンロウスカイ,セ5,58.0,横山典弘,栗東松永幹夫,NaN,12,2019105546,00660,01092,2,5,12,58.0,2
2,2,3,ステラヴェローチェ,牡6,58.0,佐々木大輔,栗東須貝尚介,NaN,13,2018104980,01197,01105,0,6,13,58.0,2
3,3,4,タスティエーラ,牡4,58.0,松山弘平,美浦堀宣行,NaN,7,2020103532,01126,01070,0,4,7,58.0,3
4,3,5,ノースブリッジ,牡6,58.0,岩田康誠,美浦奥村武,NaN,10,2018102348,05203,01145,0,6,10,58.0,3
5,4,6,ソールオリエンス,牡4,58.0,横山武史,美浦手塚貴久,NaN,5,2020102899,01170,01038,0,4,5,58.0,4
6,4,7,ドウデュース,牡5,58.0,武豊,栗東友道康夫,NaN,3,2019105283,00666,01061,0,5,3,58.0,4
7,5,8,キングズパレス,牡5,58.0,Ａ．シュタルケ,美浦戸田博文,NaN,9,2019104828,05115,01054,0,5,9,58.0,5
8,5,9,ホウオウビスケッツ,牡4,58.0,岩田望来,美浦奥村武,NaN,8,2020103941,01174,01145,0,4,8,58.0,5
9,6,10,ダノンベルーガ,牡5,58.0,Ｃ．デムーロ,美浦堀宣行,NaN,11,2019105195,05473,01070,0,5,11,58.0,6


In [12]:
sex_mapping = {"牡": 0, "牝": 1, "セ": 2}
race_type_mapping = {"ダ": 0, "芝": 1, "障": 3}
weather_mapping = {"晴": 0, "曇": 1, "小雨": 2, "雨": 3}
ground_state_mapping = {"良": 0, "稍": 1, "稍重": 1, "重": 2, "不":3, "不良":3}
race_class_mapping = {
    "新馬": 0,
    "未勝利": 1,
    "1勝クラス": 2,
    "2勝クラス": 3,
    "3勝クラス": 4,
    "オープン": 5,
    "G3": 6,
    "G2": 7,
    "G1": 8,
    "OP": 5,
    "特別": 5,
    "500万以下": 2,
    "1000万以下": 3,
    "1600万以下": 4,
    }

df["sex"] = df["性齢"].str[0].map(sex_mapping)
df["age"] = df["性齢"].str[1:].astype(int)
# df["weight"] = df["馬体重(増減)"].str.extract(r"(\d+)").astype(int)
# df["weight"] = pd.to_numeric(df["weight"], errors="coerce")
# df["weight_diff"] = df["馬体重(増減)"].str.extract(r"\((.+)\)")
# df["weight_diff"] = df["weight_diff"].fillna(0).astype(int)
# df["weight_diff"] = pd.to_numeric(df["weight_diff"], errors="coerce")
df["popularity"] = df["人気"].astype(int)
df["impost"] = df["斤量"].astype(float)
df["wakuban"] = df["枠"].astype(int)
df["umaban"] = df["馬番"].astype(int)

In [15]:
df = df[
    [
        #"race_id",
        "horse_id",
        "jockey_id",
        "trainer_id",
        #"rank",
        "sex",
        "age",
        #"weight",
        #"weight_diff",
        #"tansho_odds",
        "popularity",
        "impost",
        "wakuban",
        "umaban"
    ]
]

#df.to_csv(output_dir / save_filename, sep="\t")